In [34]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [35]:
data_dir = "/Users/subhadyutirath/Documents/CataractDetectionModel/train"

In [36]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [37]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [38]:
class_names = dataset.classes
print("Classes:", class_names)

Classes: ['immature', 'mature']


In [39]:
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [40]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

RESNET-18

In [52]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes: mature, immature
model = model

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [53]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [54]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images
        labels = labels

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")



Epoch 1/15, Loss: 0.2400, Accuracy: 88.11%
Epoch 2/15, Loss: 0.0090, Accuracy: 100.00%
Epoch 3/15, Loss: 0.0011, Accuracy: 100.00%
Epoch 4/15, Loss: 0.0012, Accuracy: 100.00%
Epoch 5/15, Loss: 0.0018, Accuracy: 100.00%
Epoch 6/15, Loss: 0.0025, Accuracy: 100.00%
Epoch 7/15, Loss: 0.0008, Accuracy: 100.00%
Epoch 8/15, Loss: 0.0002, Accuracy: 100.00%
Epoch 9/15, Loss: 0.0002, Accuracy: 100.00%
Epoch 10/15, Loss: 0.0004, Accuracy: 100.00%
Epoch 11/15, Loss: 0.0003, Accuracy: 100.00%
Epoch 12/15, Loss: 0.0002, Accuracy: 100.00%
Epoch 13/15, Loss: 0.0003, Accuracy: 100.00%
Epoch 14/15, Loss: 0.0002, Accuracy: 100.00%
Epoch 15/15, Loss: 0.0007, Accuracy: 100.00%


In [55]:
torch.save(model.state_dict(), "resnet18_cataract_model.pkl")

In [56]:
model.eval()
val_preds, val_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images = images
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.numpy())

print("\nValidation Classification Report:")
print(classification_report(val_labels, val_preds, target_names=class_names))


Validation Classification Report:
              precision    recall  f1-score   support

    immature       1.00      1.00      1.00        42
      mature       1.00      1.00      1.00        40

    accuracy                           1.00        82
   macro avg       1.00      1.00      1.00        82
weighted avg       1.00      1.00      1.00        82



VGG-16

In [47]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 2)
model = vgg16

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/subhadyutirath/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [01:51<00:00, 4.95MB/s] 


In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [49]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images
        labels = labels

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch 1/15, Loss: 0.4079, Accuracy: 76.83%
Epoch 2/15, Loss: 0.0232, Accuracy: 99.09%
Epoch 3/15, Loss: 0.0014, Accuracy: 100.00%
Epoch 4/15, Loss: 0.0001, Accuracy: 100.00%
Epoch 5/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 6/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 7/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 8/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 9/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 10/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 11/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 12/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 13/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 14/15, Loss: 0.0000, Accuracy: 100.00%
Epoch 15/15, Loss: 0.0000, Accuracy: 100.00%


In [50]:
torch.save(model.state_dict(), "vgg_cataract_model.pkl")

In [51]:
model.eval()
val_preds, val_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images = images
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.numpy())

print("\nValidation Classification Report:")
print(classification_report(val_labels, val_preds, target_names=class_names))


Validation Classification Report:
              precision    recall  f1-score   support

    immature       1.00      1.00      1.00        42
      mature       1.00      1.00      1.00        40

    accuracy                           1.00        82
   macro avg       1.00      1.00      1.00        82
weighted avg       1.00      1.00      1.00        82



In [57]:
densenet = models.densenet121(pretrained=True)
densenet.classifier = nn.Linear(densenet.classifier.in_features, 2)
model = densenet


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/subhadyutirath/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:05<00:00, 5.62MB/s]


In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [59]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images
        labels = labels

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch 1/15, Loss: 0.3867, Accuracy: 85.37%
Epoch 2/15, Loss: 0.0428, Accuracy: 100.00%
Epoch 3/15, Loss: 0.0149, Accuracy: 100.00%
Epoch 4/15, Loss: 0.0072, Accuracy: 100.00%
Epoch 5/15, Loss: 0.0121, Accuracy: 100.00%
Epoch 6/15, Loss: 0.0056, Accuracy: 100.00%
Epoch 7/15, Loss: 0.0031, Accuracy: 100.00%
Epoch 8/15, Loss: 0.0028, Accuracy: 100.00%
Epoch 9/15, Loss: 0.0021, Accuracy: 100.00%
Epoch 10/15, Loss: 0.0027, Accuracy: 100.00%
Epoch 11/15, Loss: 0.0020, Accuracy: 100.00%
Epoch 12/15, Loss: 0.0017, Accuracy: 100.00%
Epoch 13/15, Loss: 0.0016, Accuracy: 100.00%
Epoch 14/15, Loss: 0.0015, Accuracy: 100.00%
Epoch 15/15, Loss: 0.0018, Accuracy: 100.00%


In [60]:
torch.save(model.state_dict(), "densenet121_cataract_model.pkl")

In [61]:
model.eval()
val_preds, val_labels = [], []

with torch.no_grad():
    for images, labels in val_loader:
        images = images
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.numpy())

print("\nValidation Classification Report:")
print(classification_report(val_labels, val_preds, target_names=class_names))


Validation Classification Report:
              precision    recall  f1-score   support

    immature       1.00      1.00      1.00        42
      mature       1.00      1.00      1.00        40

    accuracy                           1.00        82
   macro avg       1.00      1.00      1.00        82
weighted avg       1.00      1.00      1.00        82

